In [52]:
using OceananigansLagrangianFilter
using Printf
using JLD2
using Oceananigans.Units
using Oceananigans.Units: Time
using CairoMakie
using CUDA
using Oceananigans.TimeSteppers: reset!



In [53]:
# User defined options

original_data_filename = joinpath(@__DIR__, "periodic_adjustment.jld2")
T_start = 0
T_end = 5days

arch = GPU()

# Set the output period
T_out = 1hour

# Set filter order and cut-off frequency
# Amplitude of frequency response of filter will be squared Butterworth order 2^N
N = 2 
freq_c = 1e-4/2

# Define variables to filter
original_var_names = ("wc","T", "b")

# Define velocities to use for filtering
velocity_names = ("u","w")

# Set filtering parameters (this is for Butterworth-type, could define others here)
filter_params = set_BW_filter_params(N=N,freq_c=freq_c)

# Set the time step for the simulation
Δt = 20minutes

# Decide whether to solve for and output maps to generalised Lagrangian mean
map_to_mean = true

# Name output files
forward_output_filename = joinpath(@__DIR__, "forward_LF.jld2")
backward_output_filename = joinpath(@__DIR__, "backward_LF.jld2")
combined_output_filename = joinpath(@__DIR__, "combined_LF.jld2")

# Manipulate data on disk to have correct order 
T = set_data_on_disk!(original_data_filename, direction="forward", T_start = T_start, T_end = T_end)

Current direction is forward
No need to reverse order of data


432000.0

In [54]:
# Load in saved data from simulation
saved_velocities, saved_original_vars, grid = load_data(original_data_filename, original_var_names, velocity_names, architecture=arch, backend=InMemory(4))
println("Loaded data from $original_data_filename")

# Create the original variables - these will be auxiliary fields in the model
original_vars = create_original_vars(original_var_names, grid)
println("Created original variables: ", original_vars)

# Create the filtered variables - these will be tracers in the model
filtered_vars = create_filtered_vars(original_var_names, velocity_names, filter_params, map_to_mean=map_to_mean)
println("Created filtered variables: ", filtered_vars)

# Create forcing for these filtered variables
forcing = create_forcing(filtered_vars, original_var_names, velocity_names, filter_params)
println("Created forcing for filtered variables ")

# Define model 
model = LagrangianFilter(;grid, tracers = filtered_vars, auxiliary_fields = original_vars, forcing = forcing)
println("Created model")

# Define our outputs # This seems to take a while
filtered_outputs = create_output_fields(model, original_var_names, velocity_names, filter_params)
println("Defined outputs")

# Define the filtering simulation 
simulation = Simulation(model, Δt = Δt, stop_time = T) 
println("Defined simulation")

# Tell the simulation to use the saved data
simulation.callbacks[:update_input_data] = Callback(update_input_data!, parameters = (velocities = saved_velocities, original_vars = saved_original_vars))

# Add a progress monitor
function progress(sim)
    @info @sprintf("Simulation time: %s, max(|u|):%.2e \n", 
                   prettytime(sim.model.clock.time), 
                   maximum(abs, model.velocities.u))             
     return nothing
end

simulation.callbacks[:progress] = Callback(progress, IterationInterval(50))

# Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = forward_output_filename,
                                                        schedule = TimeInterval(T_out),
                                                        overwrite_existing = true)

run!(simulation)

Loaded data from /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/geostrophic_adjustment/periodic_adjustment.jld2
Created original variables: NamedTuple with 3 Fields on 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo:
├── T: 300×1×80 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
├── b: 300×1×80 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
└── wc: 300×1×80 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
Created filtered variables: (:wcC1, :wcC2, :TC1, :TC2, :bC1, :bC2, :xi_u_C1, :xi_u_C2, :xi_w_C1, :xi_w_C2, :wcS1, :wcS2, :TS1, :TS2, :bS1, :bS2, :xi_u_S1, :xi_u_S2, :xi_w_S1, :xi_w_S2)
Created forcing for filtered variables 
Created model
Defined outputs
Defined simulation


[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds, max(|u|):0.00e+00 
[ Info:     ... simulation initialization complete (3.785 minutes)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (19.344 seconds).
[ Info: Simulation time: 16.667 hours, max(|u|):1.06e-02 
[ Info: Simulation time: 1.389 days, max(|u|):1.91e-02 
[ Info: Simulation time: 2.083 days, max(|u|):2.50e-02 
[ Info: Simulation time: 2.778 days, max(|u|):2.82e-02 
[ Info: Simulation time: 3.472 days, max(|u|):2.97e-02 
[ Info: Simulation time: 4.167 days, max(|u|):2.86e-02 
[ Info: Simulation time: 4.861 days, max(|u|):2.42e-02 
[ Info: Simulation is stopping after running for 4.275 minutes.
[ Info: Simulation time 5 days equals or exceeds stop time 5 days.


In [55]:

# Now, run it backwards. Switch the data direction on disk
T = set_data_on_disk!(original_data_filename, direction="backward", T_start = T_start, T_end = T_end)

# Reload the saved data 
saved_velocities, saved_original_vars, grid = load_data(original_data_filename, original_var_names, velocity_names, architecture=arch, backend=InMemory(4))

# Reset the filtering simulation 
reset!(model.clock)

# Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = backward_output_filename,
                                                        schedule = TimeInterval(T_out),
                                                        overwrite_existing = true)

# And run the backward simulation.
run!(simulation)


Current direction is forward
Reversing order of data
Reversed order of b
Reversed order of and switched sign of u
Reversed order of and switched sign of v
Reversed order of and switched sign of w
Reversed order of wc
Reversed order of T
Reversed order of t_simulation
Reversed order of and shifted t
New direction is backward


[ Info: Initializing simulation...
[ Info: Simulation time: 0 seconds, max(|u|):2.56e-02 
[ Info:     ... simulation initialization complete (39.339 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (12.958 ms).
[ Info: Simulation time: 16.667 hours, max(|u|):1.89e-02 
[ Info: Simulation time: 1.389 days, max(|u|):1.06e-02 
[ Info: Simulation time: 2.083 days, max(|u|):6.39e-03 
[ Info: Simulation time: 2.778 days, max(|u|):1.12e-02 
[ Info: Simulation time: 3.472 days, max(|u|):1.73e-02 
[ Info: Simulation time: 4.167 days, max(|u|):2.40e-02 
[ Info: Simulation time: 4.861 days, max(|u|):2.84e-02 
[ Info: Simulation is stopping after running for 8.745 seconds.
[ Info: Simulation time 5 days equals or exceeds stop time 5 days.


In [56]:
sum_forward_backward_contributions!(combined_output_filename, forward_output_filename, backward_output_filename, T, velocity_names, original_var_names)

Combined forward and backward contributions into /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/geostrophic_adjustment/combined_LF.jld2


In [49]:
# Next step is to interpolate onto the correct grid
using GeoStats: Polynomial, NN, Kriging, GaussianVariogram
using Oceananigans.BoundaryConditions: PeriodicBoundaryCondition
using GeoStats: georef, InterpolateNeighbors, RegularGrid, IDW, Euclidean, LWR
using JLD2: Group
model = Kriging(GaussianVariogram(range=100))
#model = Polynomial(5)
data_array = regrid_to_mean_position!(combined_output_filename, original_var_names, velocity_names, model, 40,20)


0.0%┣                                              ┫ 0/121 [00:00<00:00, -0s/it]
0.8%┣▍                                         ┫ 1/121 [00:02<Inf:Inf, InfGs/it]
1.7%┣▊                                              ┫ 2/121 [00:04<07:17, 4s/it]
2.5%┣█▏                                             ┫ 3/121 [00:05<05:24, 3s/it]
3.3%┣█▌                                             ┫ 4/121 [00:07<04:44, 2s/it]
4.1%┣██                                             ┫ 5/121 [00:09<04:24, 2s/it]
5.0%┣██▎                                            ┫ 6/121 [00:11<04:11, 2s/it]
5.8%┣██▊                                            ┫ 7/121 [00:13<04:02, 2s/it]
6.6%┣███                                            ┫ 8/121 [00:15<03:55, 2s/it]
7.4%┣███▌                                           ┫ 9/121 [00:16<03:50, 2s/it]
8.3%┣███▉                                          ┫ 10/121 [00:18<03:45, 2s/it]
9.1%┣████▏                                         ┫ 11/121 [00:20<03:41, 2s/it]
9.9%┣████▋                  

In [57]:
jld2_to_netcdf(combined_output_filename, joinpath(@__DIR__, "combined_output.nc"))
#jld2_to_netcdf(forward_output_filename, joinpath(@__DIR__, "forward_output.nc"))

closed Dataset

In [51]:
# Animate
timeseries1 = FieldTimeSeries(combined_output_filename, "b")
timeseries2 = FieldTimeSeries(combined_output_filename, "b_filtered")
timeseries3 = FieldTimeSeries(combined_output_filename, "b_filtered_regrid")

timeseries4 = FieldTimeSeries(combined_output_filename, "xi_u")
timeseries5 = FieldTimeSeries(combined_output_filename, "xi_w")
timeseries6 = FieldTimeSeries(combined_output_filename, "b_filtered_regrid")

times = timeseries1.times

set_theme!(Theme(fontsize = 20))
fig = Figure(size = (1000, 500))

axis_kwargs = (xlabel = "x",
               ylabel = "z",
               aspect = AxisAspect(1))

ax1 = Axis(fig[2, 1]; title = "Raw ", axis_kwargs...)
ax2 = Axis(fig[2, 2]; title = "Filtered ", axis_kwargs...)
ax3 = Axis(fig[2, 3]; title = "Filtered regridded ", axis_kwargs...)


n = Observable(1)


var1 = @lift timeseries1[$n]
var2 = @lift timeseries2[$n]
var3 = @lift timeseries3[$n]

var4 = @lift timeseries4[$n]
var5 = @lift timeseries5[$n]
var6 = @lift timeseries6[$n]

vmin = -1e-4
vmax = 1e-4
heatmap!(ax1, var1; colormap = :balance, colorrange = (vmin, vmax))
heatmap!(ax2, var2; colormap = :balance, colorrange = (vmin, vmax))
heatmap!(ax3, var3; colormap = :balance, colorrange = (vmin, vmax))
contour!(ax1, var1 ; levels=20, color=:black, linewidth=1)
contour!(ax2, var2 ; levels=20, color=:black, linewidth=1)
contour!(ax3, var3 ; levels=20, color=:black, linewidth=1)


title = @lift "t = " * string(round(times[$n], digits=2))
Label(fig[1, 1:3], title, fontsize=24, tellwidth=false)

fig

frames = 1:length(times)

@info "Making an animation"

CairoMakie.record(fig, "LF_movie_b_improved_interp.mp4", frames, framerate=24) do i
    n[] = i
end

[ Info: Making an animation


"LF_movie_b_improved_interp.mp4"

In [ ]:
scpy = pyimport("scipy")
scpy.interpolate.LinearNDInterpolator